In [73]:
import pandas as pd
from pandasql import sqldf
import pyparsing as pyp
from os import path
import re
import string

# TODO Solve this problem with the relative paths.
INTX_PATH = '../data/clean/Jefferson_County_KY_Street_Intersections.csv'
assert path.exists(INTX_PATH)
INTX = pd.read_csv(INTX_PATH, index_col=0)

SIG_PATH = "../data/clean/Jefferson_County_KY_Signalized_Intersections.csv"
assert path.exists(SIG_PATH)
SIG = pd.read_csv(SIG_PATH, index_col=0)

display("INTX", INTX.head(3), "SIG", SIG.head(3))

'INTX'

,intid,main_street,cross_street,main_suffix,cross_suffix,cross_prefix,main_prefix,longitude,latitude
objectid,,,,,,,,,
1,154647662,REHL,REHL,RD,CT,W,,-85.510444,38.205887
2,254646551,REHL,TUCKER STATION,RD,RD,,,-85.528169,38.200376
3,354646551,REHL,TUCKER STATION,RD,RD,,,-85.528419,38.200360


'SIG'

,main_street,cross_street,longitude,latitude,description
intid,,,,,
76747569996,OLD HENRY,I 265 RAMP,-85.499337,38.270632,OLD HENRY @ I 265 RAMP
78959406728,SHELBYVILLE,BECKLEY WOODS,-85.498479,38.241589,SHELBYVILLE @ BECKLEY WOODS
123759407630,SHELBYVILLE,LAKE FOREST,-85.493550,38.240517,SHELBYVILLE @ LAKE FOREST


In [77]:

def name_splitter(name):
    chunks = name.split()
    assert len(chunks) > 0
    if len(chunks) == 1:
        onset = coda = pd.NA
        nucleus = chunks
    else:
        [onset, *nucleus, coda] = chunks
    if not nucleus:
        nucleus = pd.NA
    return onset, nucleus, coda


split = pd.DataFrame.from_records(SIG.main_street.apply(name_splitter).values, 
                          index=SIG.index, columns="onset center final".split())


In [87]:

do = lambda f:f()

@set
@do
def suffixes():
    intersection_suffixes = INTX.main_suffix.unique()
    for suff in split.final.dropna().unique():
        if suff in intersection_suffixes:
            yield suff

suffixes


{'AVE', 'HWY', 'LOOP', 'PKY', 'RD', 'RUN', 'ST'}

In [88]:

intersection_prefixes = INTX.main_prefix.unique()
intersection_prefixes

prefixes = set("NSEW")
prefixes

{'E', 'N', 'S', 'W'}

In [90]:
def new_splitter(name):
    chunks = name.split()
    assert len(chunks) > 0
    out = {x:pd.NA for x in "onset nucleus coda".split()}

    if len(chunks) == 1:
        out['nucleus'] = chunks
    else:
        [onset, *nucleus, coda] = chunks

    if coda not in prefixes:
        nucleus.append(coda)
    return out

In [ ]:
def get_all_roads(df:pd.DataFrame, series_name) -> pd.Series:
    return set(pd.concat((df.main_street, df.cross_street), ignore_index=True))



names_SIG = get_all_roads(SIG, "signal names")
names_INTX = get_all_roads(INTX, "interx names")

perfect_matches = names_INTX.intersection(names_SIG)
perfect_matches


{'10TH',
 '11TH',
 '12TH',
 '12TH STREET CONNECTOR',
 '13TH',
 '15TH',
 '16TH',
 '17TH',
 '18TH',
 '19TH',
 '1ST',
 '21ST',
 '22ND',
 '24TH',
 '26TH',
 '28TH',
 '29TH',
 '2ND',
 '30TH',
 '31ST',
 '32ND',
 '33RD',
 '34TH',
 '35TH',
 '37TH',
 '38TH',
 '3RD',
 '3RD STREET',
 '41ST',
 '4TH',
 '5TH',
 '6TH',
 '7TH',
 '7TH STREET',
 '8TH',
 '9TH',
 '9TH STREET RAMP',
 'ABRAHAM FLEXNER',
 'ADAMS',
 'ADELE',
 'AIKEN',
 'AIR COMMERCE',
 'ALDER',
 'ALFRESCO',
 'ALGONQUIN',
 'ALLMOND',
 'ALTA VISTA',
 'AMBROSSE',
 'AMY',
 'ANNA',
 'ANTLE',
 'APPLE VALLEY',
 'APPLEGATE',
 'ARCADE',
 'ARMORY',
 'ARNOLDTOWN',
 'ASHBOTTOM',
 'ASHBY',
 'ASHLAND',
 'ASHMOOR',
 'AUDUBON',
 'AUDUBON PLAZA',
 'BALLARDSVILLE',
 'BANK',
 'BARBOUR',
 'BARDSTOWN',
 'BARRET',
 'BARRETT',
 'BASHFORD MANOR',
 'BATES',
 'BAUER',
 'BAXTER',
 'BAYLY',
 'BECKLEY STATION',
 'BECKLEY WOODS',
 'BEECH',
 'BEECHWOOD',
 'BELLAIRE',
 'BELLARMINE',
 'BELLS',
 'BELMAR',
 'BENNETT',
 'BERNHEIM',
 'BERRY',
 'BETHANY',
 'BEULAH CHURCH',
 'BICKN

In [ ]:
from difflib import SequenceMatcher
import string

no_match = list(names_SIG.difference(perfect_matches))

prefixes = "NSEW"

def parse_interstate(name):
    i, number, *a = name.split()
    assert i == "I"
    return {'name':' '.join((i, name)), "suffix":' '.join(a)}

def find(name, threshold=0.6):
    matches = dict()
    for other in names_INTX:
        m = SequenceMatcher(lambda x:x in string.whitespace, name, other).ratio()
        if m >= threshold:
            matches[other] = m
    return name, matches

find(no_match[3])



('MAIN ST',
 {'MAR VISTA': 0.625,
  'IMPATIENS': 0.625,
  'MARINE': 0.6153846153846154,
  'AXMINSTER': 0.625,
  'MAVIS': 0.6666666666666666,
  'MISTY': 0.6666666666666666,
  'MANSE': 0.6666666666666666,
  'BLAZING STAR': 0.631578947368421,
  'TRAIN STATION': 0.6,
  'MADISON TRACE': 0.6,
  'MARIS': 0.6666666666666666,
  'ADKINS': 0.6153846153846154,
  'MARINA': 0.6153846153846154,
  'MARTIN HOUSE': 0.631578947368421,
  'MARIEMONT': 0.625,
  'BAINES': 0.6153846153846154,
  'MARION': 0.6153846153846154,
  'MARIAN': 0.6153846153846154,
  'MAIN': 0.7272727272727273,
  'PLAINS': 0.6153846153846154,
  'MARLIN': 0.6153846153846154,
  'MARVIN': 0.6153846153846154,
  'MAIDEN': 0.6153846153846154,
  'MORNING STAR': 0.631578947368421,
  'FALLING STAR': 0.631578947368421,
  'MARTIN': 0.6153846153846154,
  'MANSON': 0.6153846153846154,
  'MOUNTAIN ASH': 0.631578947368421})

In [ ]:
w = pyp.Or(("ab", "bc"))
w('x').parse_string("abbcabc")

ParseResults(['ab'], {'x': 'ab'})

In [ ]:
names_INTX

,interx names
0,10TH
1,11TH
2,12TH
3,12TH STREET CONNECTOR
4,13TH
...,...
9292,ZIX
9293,ZOELLER
9294,ZONETON
9295,ZORN
